### Data ###

In [ ]:
import xray

In [ ]:
data = xray.open_dataset('tracks.20130125.nc', decode_cf=False)

In [ ]:
# Dataset Information
print data

### Multi-Index ###

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ds = data.to_dataframe()

In [ ]:
ds.tail(4)

In [ ]:
# Dimension Setting
# All
M = max(ds.track)
N = len(ds.n)

In [ ]:
# Dimension Setting
# Test
# M = 3000
# N = 41320

In [ ]:
ds = ds.head(N)

In [ ]:
ds.tail(4)

In [ ]:
track = np.asarray(ds.track)
n = np.asarray(ds.n)

In [ ]:
# Multi-Index Construction
arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['Eddy ID', 'Point ID'])

In [ ]:
j1 = np.asarray(ds.j1)
cyc = np.asarray(ds.cyc)
lon = np.asarray(ds.lon)
lat = np.asarray(ds.lat)
A = np.asarray(ds.A)
L = np.asarray(ds.L)
U = np.asarray(ds.U)

In [ ]:
var = np.zeros((N, 7))

In [ ]:
for i in range(N):
    var[i][0] = j1[i]
    var[i][1] = cyc[i]
    var[i][2] = lon[i]
    var[i][3] = lat[i]
    var[i][4] = A[i]
    var[i][5] = L[i]
    var[i][6] = U[i]

In [ ]:
df = pd.DataFrame(var, index=multi_index)
df.columns = ['j1', 'cyc', 'lon', 'lat', 'A', 'L', 'U']

In [ ]:
df.tail(4)

### Time ###

In [ ]:
import datetime, jdcal

In [ ]:
def jday_to_datetime(jday, refday=0):
    y, m, d, f = jdcal.jd2gcal(jday, refday)
    h = int(f*24)
    return pd.to_datetime(datetime.datetime(y, m, d, h))

In [ ]:
pd_date = df.j1.apply(jday_to_datetime)

In [ ]:
df.j1 = pd_date
df = df.rename(columns = {'j1': 'date'})

In [ ]:
df.tail(4)

### Longitude ###

In [ ]:
# Longitude Range Setting
# From -180 to 180
lon_fix_01 = df.where(df.lon <= 540).lon - 360
lon_fix_02 = df.where(df.lon > 540).lon - 720
lon_fix_A = lon_fix_01
lon_fix_A = lon_fix_A.fillna(lon_fix_02)
df.lon = lon_fix_A

In [ ]:
# Longitude Range Setting
# From 0 to 360
lon_fix_03 = df.where(df.lon < 0).lon + 360
lon_fix_04 = df.where(df.lon >= 0).lon
lon_fix_B = lon_fix_03
lon_fix_B = lon_fix_B.fillna(lon_fix_04)
df.lon = lon_fix_B

In [ ]:
df.tail(4)

### Point ###

In [ ]:
count = ds.track
count = count.value_counts(normalize=False, sort=True, ascending=True, bins=None, dropna=False).reindex(range(1, M+1))
count = pd.DataFrame(count)
count.columns = ['point']

In [ ]:
count.tail(4)

In [ ]:
lon = np.asarray(df.lon)
lat = np.asarray(df.lat)
point = np.asarray(count.point)

In [ ]:
# Origin Coordinates
lon_o = np.zeros(M)
lat_o = np.zeros(M)

In [ ]:
# Termination Coordinates
lon_t = np.zeros(M)
lat_t = np.zeros(M)

In [ ]:
c = 0
i = 0

while i < N:
    c = int(c) + 1
    lon_o[c-1] = lon[i]
    lat_o[c-1] = lat[i]
    i = i + int(point[c-1])
    lon_t[c-1] = lon[i-1]
    lat_t[c-1] = lat[i-1]

In [ ]:
lon_o = pd.DataFrame(lon_o)
lat_o = pd.DataFrame(lat_o)
lon_t = pd.DataFrame(lon_t)
lat_t = pd.DataFrame(lat_t)

In [ ]:
# Index Construction
index = np.zeros(M)

In [ ]:
for i in range(M):
    index[i] = i+1

In [ ]:
index = pd.DataFrame(index)
index = index.astype(int)

In [ ]:
# Start Centers
sc = pd.concat([index, lon_o, lat_o], axis=1)
sc.columns = ['Eddy ID', 'lon', 'lat']
sc = sc.set_index('Eddy ID')

In [ ]:
sc.tail(4)

In [ ]:
# End Centers
ec = pd.concat([index, lon_t, lat_t], axis=1)
ec.columns = ['Eddy ID', 'lon', 'lat']
ec = ec.set_index('Eddy ID')

In [ ]:
ec.tail(4)

### LineString ###

In [ ]:
ls = np.zeros((N, 2))

In [ ]:
for i in range(N):
    ls[i][0] = lon[i]
    ls[i][1] = lat[i]

In [ ]:
ls = pd.DataFrame(ls, index=multi_index, columns=['lon', 'lat'])

In [ ]:
ls.tail(4)

### Polygon ###

In [ ]:
from numpy import cos, pi, sin

In [ ]:
L = np.asarray(df.L)

In [ ]:
L_o = np.zeros(M)
L_t = np.zeros(M)

In [ ]:
c = 0
i = 0

while i < N:
    c = int(c) + 1
    L_o[c-1] = L[i]
    i = i + int(point[c-1])
    L_t[c-1] = L[i-1]

In [ ]:
L_o = pd.DataFrame(L_o)
L_t = pd.DataFrame(L_t)

In [ ]:
# Origin Circles
oc = pd.concat([lon_o, lat_o, L_o], axis=1)
oc.columns = ['lon', 'lat', 'L']

In [ ]:
oc.tail(4)

In [ ]:
# Termination Circles
tc = pd.concat([lon_t, lat_t, L_t], axis=1)
tc.columns = ['lon', 'lat', 'L']

In [ ]:
tc.tail(4)

In [ ]:
# Circle Center Number
C = M

In [ ]:
# Circle Arc Number
A = C*17

In [ ]:
center = np.zeros(A)
arc = np.zeros(A)

In [ ]:
c = 1
i = 0
j = 1

while i < A:
    while c <= 17:
        center[i] = j
        i = i+1
        c = c+1
    j = j+1
    c = 1

In [ ]:
center = center.astype(int)

In [ ]:
c = 1
i = 0
j = 1

while i < A:
    while c <= 17:
        arc[i] = j
        i = i+1
        j = j+1
        c = c+1
    j = 1
    c = 1

In [ ]:
arc = arc.astype(int)

In [ ]:
# Multi-Index Construction
arrays = [center, arc]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['center', 'arc'])

In [ ]:
# Earth Radius in Kilometers
R = 6371

__Origins__

In [ ]:
lon = np.asarray(oc.lon)
lat = np.asarray(oc.lat)
L = np.asarray(oc.L)

In [ ]:
theta = np.zeros(C)
x = np.zeros(C)
y = np.zeros(C)
r = np.zeros(C)

In [ ]:
for i in range(C):
    theta[i] = lat[i]*(pi/180)
    r[i] = R*cos(theta[i])
    x[i] = (L[i]/r[i])*(180/pi)
    y[i] = (L[i]/R)*(180/pi)

In [ ]:
op_lon = np.zeros(A)
op_lat = np.zeros(A)
op_x = np.zeros(A)
op_y = np.zeros(A)

In [ ]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        op_lon[i] = lon[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        op_lat[i] = lat[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        op_x[i] = x[c]*cos((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        op_y[i] = y[c]*sin((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
op = np.zeros((A, 2))

In [ ]:
for i in range(A):
    op[i][0] = op_lon[i]+op_x[i]
    op[i][1] = op_lat[i]+op_y[i]

In [ ]:
# Origin Polygons
op = pd.DataFrame(op, index=multi_index)
op.columns = ['lon', 'lat']

In [ ]:
op.tail(4)

__Terminations__

In [ ]:
lon = np.asarray(tc.lon)
lat = np.asarray(tc.lat)
L = np.asarray(tc.L)

In [ ]:
theta = np.zeros(C)
r = np.zeros(C)
x = np.zeros(C)
y = np.zeros(C)

In [ ]:
for i in range(C):
    theta[i] = lat[i]*(pi/180)
    r[i] = R*cos(theta[i])
    x[i] = (L[i]/r[i])*(180/pi)
    y[i] = (L[i]/R)*(180/pi)

In [ ]:
tp_lon = np.zeros(A)
tp_lat = np.zeros(A)
tp_x = np.zeros(A)
tp_y = np.zeros(A)

In [ ]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        tp_lon[i] = lon[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        tp_lat[i] = lat[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        tp_x[i] = x[c]*cos((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        tp_y[i] = y[c]*sin((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [ ]:
tp = np.zeros((A, 2))

In [ ]:
for i in range(A):
    tp[i][0] = tp_lon[i]+tp_x[i]
    tp[i][1] = tp_lat[i]+tp_y[i]

In [ ]:
# Termination Polygons
tp = pd.DataFrame(tp, index=multi_index)
tp.columns = ['lon', 'lat']

In [ ]:
tp.tail(4)

### ID ###

In [ ]:
eddy_id = pd.DataFrame(index)
eddy_id = pd.concat([index, index], axis=1)
eddy_id.columns = ['Eddy ID', 'eddy id']
eddy_id = eddy_id.set_index('Eddy ID')

In [ ]:
eddy_id.tail(4)

### Date ###

In [ ]:
ds.tail(4)

In [ ]:
j1 = np.asarray(ds.j1)

In [ ]:
j1_o = np.zeros(M)
j1_t = np.zeros(M)

In [ ]:
c = 0
i = 0

while i < N:
    c = int(c) + 1
    j1_o[c-1] = j1[i]
    i = i + int(point[c-1])
    j1_t[c-1] = j1[i-1]

In [ ]:
j1_o = pd.DataFrame(j1_o)
j1_t = pd.DataFrame(j1_t)

In [ ]:
# Start Julian Dates
sj = pd.concat([index, j1_o], axis=1)
sj.columns = ['Eddy ID', 'j1']
sj = sj.set_index('Eddy ID')

In [ ]:
sj.tail(4)

In [ ]:
# End Julian Dates
ej = pd.concat([index, j1_t], axis=1)
ej.columns = ['Eddy ID', 'j1']
ej = ej.set_index('Eddy ID')

In [ ]:
ej.tail(4)

In [ ]:
# Start Dates
sd = sj.j1.apply(jday_to_datetime)
sd = pd.DataFrame(sd)
sd = sd.rename(columns = {'j1': 'date'})

In [ ]:
sd.tail(4)

In [ ]:
# End Dates
ed = ej.j1.apply(jday_to_datetime)
ed = pd.DataFrame(ed)
ed = ed.rename(columns = {'j1': 'date'})

In [ ]:
ed.tail(4)

In [ ]:
# Durations in Days
dd = ed-sd

In [ ]:
dd.date.max()

In [ ]:
dd.tail(4)

In [ ]:
# Durations in Integers
di = np.zeros(M)

In [ ]:
for i in range(M):
    di[i] = np.timedelta64(dd.date[i+1], 'D')/np.timedelta64(1, 'D')

In [ ]:
di = pd.DataFrame(di)

In [ ]:
di = pd.concat([index, di], axis=1)
di.columns = ['Eddy ID', 'day']
di = di.set_index('Eddy ID')
di.day = di.day.astype(int)

In [ ]:
di.tail(4)

### Area ###

In [ ]:
radius = np.asarray(ds.L)

In [ ]:
area = 2*pi*(radius**2)

In [ ]:
aa = np.zeros(M)

In [ ]:
i = 0
c = 0

while i < N:
    c = int(c) + 1
    aa[c-1] = area[i]
    i = i + int(point[c-1])

In [ ]:
aa = pd.DataFrame(aa)

In [ ]:
# Area in Square Kilometers
aa = pd.concat([index, aa], axis=1)
aa.columns = ['Eddy ID', 'area']
aa = aa.set_index('Eddy ID')

# Area in Square Meters
aa = aa*10**6

In [ ]:
aa.tail(4)

### Vorticity ###

In [ ]:
from tqdm import tqdm

In [ ]:
# Relative Vorticity
zeta = np.asarray((ds.U*0.01)/(ds.L*1000))

In [ ]:
# Multi-Index Construction
arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['Eddy ID', 'Point ID'])

In [ ]:
zeta = pd.DataFrame(zeta, index=multi_index)
zeta.columns = ['relative vorticity']

In [ ]:
zeta.tail(4)

In [ ]:
lav = np.zeros(M)

In [ ]:
for i in tqdm(range(0, M)):
    lav[i] = zeta.loc[i+1].mean()

In [ ]:
lav = pd.DataFrame(lav)

In [ ]:
lav = pd.concat([index, lav], axis=1)
lav.columns = ['Eddy ID', 'zeta']
lav = lav.set_index('Eddy ID')

In [ ]:
lav.tail(4)

### GeoJSON ###

In [ ]:
from geojson import LineString, Point, Polygon
from pymongo import MongoClient

In [ ]:
client = MongoClient()
db = client.eddies

In [ ]:
# Print Database Names
client.database_names()

In [ ]:
# Print Collection Names
db.collection_names()

In [ ]:
# Removes Documents from the Collection
result = db.ssh_eddies.remove()

In [ ]:
# Inserts Documents into the Collection
for i in tqdm(range(1, M+1)):

    # GeoJSON
    eddy = {
        '_id': int(eddy_id.loc[i]),
        'type': 'FeatureSet',
        'loc_start': [sc.loc[i]['lon'], sc.loc[i]['lat']],
        'loc_end': [ec.loc[i]['lon'], ec.loc[i]['lat']],
        'date_start': sd.loc[i]['date'],
        'date_end': ed.loc[i]['date'],
        'duration': di.loc[i]['day'],
        'area': aa.loc[i]['area'],
        'lav': lav.loc[i]['zeta'],
        'features': [
            {
                'type': 'Feature',
                'properties': {'name': 'start_center'},
                'geometry': Point(tuple(sc.loc[i][['lon', 'lat']].values))
            },
            {
                'type': 'Feature',
                'properties': {'name': 'end_center'},
                'geometry': Point(tuple(ec.loc[i][['lon', 'lat']].values))
            },
            {
                'type': 'Feature',
                'properties': {'name': 'trajectory'},
                'geometry': LineString([tuple(x) for x in ls.loc[i][['lon', 'lat']].values])
            },
            {
                'type': 'Feature',
                'properties': {'name': 'start_polygon'},
                'geometry': Polygon([[tuple(x) for x in op.loc[i][['lon', 'lat']].values]])
            },
            {
                'type': 'Feature',
                'properties': {'name': 'end_polygon'},
                'geometry': Polygon([[tuple(x) for x in tp.loc[i][['lon', 'lat']].values]])
            }
        ]    
    }
    
    # MongoDB
    result = db.ssh_eddies.insert_one(dict(eddy))